# Titanic Machine Learning and AIX360 Demonstrator

## To know more about AIX360 toolkit: https://github.com/Trusted-AI/AIX360

In [1]:

# We need to install a specific version of AIX360 for stability
!pip install aix360==0.2.1
!pip uninstall shap --yes --no-cache -q
!pip install shap==0.34 --no-cache -q
!pip uninstall pandas --yes --no-cache -q
!pip install pandas==1.1.0 --no-cache -q
!pip uninstall click --yes --no-cache -q
!pip install click>=7.1.1 --no-cache -q





^C
ERROR: Operation cancelled by user
^C
ERROR: Operation cancelled by user
^C
ERROR: Operation cancelled by user
ERROR: xport 3.2.1 requires pandas>=1.0.3, which is not installed.
ERROR: aix360 0.2.0 requires pandas, which is not installed.
ERROR: aix360 0.2.0 requires shap, which is not installed.


## Now, restart the kernel and you can import directly the library by runnin the cell below

## Import the libraries

In [ ]:


from aix360.algorithms.protodash import ProtodashExplainer
#from aix360.datasets.cdc_dataset import CDCDataset


%matplotlib inline


In [ ]:
import pandas as pd
import numpy as np
import datetime
import calendar
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

import seaborn as sns
import matplotlib.pyplot as plt
#sns.set_theme(style="ticks", color_codes=True)


from sklearn.preprocessing import OneHotEncoder

## Import the datasets

In [ ]:
import pandas as pd

# Load Data


url = 'https://github.com/IBMDeveloperMEA/Trusted-AI-Build-Explainable-ML-Models-using-AIX360/blob/main/dataset/titanic-complete.csv?raw=True'
df_data_1= pd.read_csv(url)


url = 'https://github.com/IBMDeveloperMEA/Trusted-AI-Build-Explainable-ML-Models-using-AIX360/blob/main/dataset/titanic-train.csv?raw=True'
training_data = pd.read_csv(url)

url = 'https://github.com/IBMDeveloperMEA/Trusted-AI-Build-Explainable-ML-Models-using-AIX360/blob/main/dataset/titanic-test.csv?raw=True'
test_data = pd.read_csv(url)
test_data.head()

In [ ]:
training_data.shape[0]

[<h2>TRAINING AND TEST DATA</h2>](#INDEX)

### TRAINING DATA

The training data consists of the 891 of the passengers in the passenger manifest for the RMS Titanic.

- Passengerid - unique sequential numebr
- Survived - binary indicator of whether they survived
- Pclass - passenger class
- Name - passenger's full name
- Sex - passenger's gender
- SibSp - how many siblings and/or spouses were with them
- Parch - how many parents or children were with them
- Ticket - their ticket number
- Fare - they amount they paid for their ticket
- Cabin - cabin number 
- Embarked - port of embarcation C = Cherbourg; Q = Queenstown; S = Southampton


In [ ]:
# Test data loaded from IBM Cloud

test_data.count()

As for the training dataset, age and cabin attributes have many missing datapoints.

## Check how many values are missing 

In [ ]:

nan = np.nan
age_missing = training_data.query("age != @nan")[['survived','passengerId','age']].groupby('survived').count()
age_missing['MISSING'] = abs(age_missing['age']-age_missing['passengerId'])
print("Total missing = ", age_missing['MISSING'].sum(), ". Percentage: ", age_missing['MISSING'].sum()*100/training_data.shape[0])


### Around 20% data from age is missing - will delete the rows as age is important

In [ ]:
training_data.groupby('survived').count()[['passengerId','cabin']]

nan = np.nan
cabin_missing = training_data.query("age != @nan")[['survived','passengerId','cabin']].groupby('survived').count()
cabin_missing['MISSING'] = abs(cabin_missing['cabin']-cabin_missing['passengerId'])
print("Total missing = ", cabin_missing['MISSING'].sum(), ". Percentage: ", cabin_missing['MISSING'].sum()*100/training_data.shape[0])

### Around 70% data from cabin is missing - will not use the column

### COMPARE TRAINING & TEST SET DISTRIBUTION FOR TARGET VARIABLE

Before moving to the modeling phase, we also verify whether the distribution of survived passengers is the same in the training and test set, to avoid biases in the predictions linked to the different distributions in the input data.

In [ ]:
fig = plt.figure(figsize = (12,5))
titles = ['TRAINING','TEST']
for i,d in enumerate([training_data, test_data]):
    ax = fig.add_subplot(1,2,i+1)
    sns.barplot(x="sex", y="survived", hue="pclass", data=d, ax = ax)
    ax.set_title = titles[i]

plt.show()

print("Survived passengers seemed to be proportionally distributed among train and test dataset; they also display similar percentage across sex and class")

[<h2>AIX360: EXPLAINABILITY OVERVIEW</h2>](#INDEX)


Know when to use what explainer

<img src="https://raw.githubusercontent.com/Trusted-AI/AIX360/master/aix360/algorithms/methods-choice-updated.png" alt="Guidance on use of AI Explainability 360 algorithms" width="900"/>



[<h2>Explainability meaning and level is different for everyone</h2>](#INDEX)
[<h3>1 - Data Scientist</h3>](#INDEX)
[<h3>2 - Ship Captain</h3>](#INDEX)
[<h3>3 - Passenger </h3>](#INDEX)


[<h2>Explanations</h2>](#INDEX)



<img src="https://miro.medium.com/max/2000/1*wd7Xd9bb7wrT0qAwi-FVew.png" alt="How to explain" width="900"/>

## 1-BRCG - BOOLEAN RULE COLUMN GENERATOR


BRCG is global explainer - useful for Data Scientists

AIX360 provides alternative explanation method which allow to build a model and - at the same time - provide explanations for the prediciton.
Here we explore BRCG - Boolean Rule Column Generator - and LinRR - Linear Rule XX.



In [ ]:
## BRCG requires features to be binarized
# Categorical features
colCateg = ['pclass']

# Binarize data and also return standardized ordinal features
from aix360.algorithms.rbm import FeatureBinarizer
fb = FeatureBinarizer(colCateg=colCateg, negations=True, returnOrd=True)

## apply binarization to all relevant features
dfTrain, dfTrainStd = fb.fit_transform(training_data[['sex','pclass','parch','sibsp']])
dfTest, dfTestStd = fb.transform(test_data[['sex','pclass','parch','sibsp']])
yTrain = training_data['survived']
yTest = test_data['survived']

In [ ]:
## We establish baseline, by building a gradient boosting model and we will then compare the results from the
## baseline model with accuracy obtained through BRCG.

# Train and evaluate GBCT
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(n_estimators=100)
gbc.fit(dfTrain, yTrain)
from sklearn.metrics import accuracy_score
print('Training accuracy:', accuracy_score(yTrain, gbc.predict(dfTrain)))
print('Test accuracy:', accuracy_score(yTest, gbc.predict(dfTest)))

In [ ]:
dfTrain

In [ ]:
# Instantiate BRCG
from aix360.algorithms.rbm import BooleanRuleCG
br = BooleanRuleCG(lambda0=1e-4, lambda1=1e-4)
# Train, print, and evaluate model
br.fit(dfTrain, yTrain)
print('Training accuracy:', accuracy_score(yTrain, br.predict(dfTrain)))
print('Test accuracy:', accuracy_score(yTest, br.predict(dfTest)))
print('Predict Y=1 if ANY of the following rules are satisfied, otherwise Y=0:')
print(br.explain()['rules'])

**RESULTS**

Accuracy obtained with BRCG model is comparable to the baseline model.
The final row output is 


*Predict Y=1 if ANY of the following rules are satisfied, otherwise Y=0:*


*['sex not  AND pclass != 3', 'sex not  AND pclass == 3 AND parch <= 2 AND sibsp <= 2']*

NOTE: sex not = FEMALE

BRCG usually builds "simpler" models, to avoid rules proliferation. In this specific case, even though we just have 2 rules as the model output, the performances are not impacted.
As demonstrated below, the two rules hold true for both train and test dataset - meaning that all positive predictions satisfy those rules and viceversa.

In [ ]:
training_data['PREDICTION'] = br.predict(dfTrain)
print("the rule provided by the model holds true for the TRAINING dataset")
training_data.query("(sex == 0 and pclass!=3) or (sex == 0 and pclass == 3 and parch <= 2 and sibsp <= 2)").groupby('PREDICTION').count()


In [ ]:
test_data['PREDICTION'] = br.predict(dfTest)
print("the rule provided by the model holds true for the TEST dataset")
test_data.query("(sex == 0 and pclass!=3) or (sex == 0 and pclass == 3 and parch <= 2 and sibsp <= 2)").groupby('PREDICTION').count()


[<h2>1- LinRR - LINEAR RULE REGRESSION</h2>](#INDEX)

In [ ]:
'''# Instantiate LinRR with good complexity penalties and numerical features
from aix360.algorithms.rbm import LinearRuleRegression
from sklearn.metrics import r2_score

lrr = LinearRuleRegression(lambda0=0.09, lambda1=0.09, useOrd=True)
# Train and evaluate model
lrr.fit(dfTrain, yTrain, dfTrainStd)
print('Training R^2:', r2_score(yTrain, lrr.predict(dfTrain, dfTrainStd)))
print('Test R^2:', r2_score(yTest, lrr.predict(dfTest, dfTestStd)))



'''
# Print model
lrr.explain()

## 2- Ship Captain - Check details, WHY should someone be saved? Is the prediction fair?

[<h2>KNOW YOUR PERSONAS: PROTODASH</h2>](#INDEX)

After analyzing the dataset, we can now apply local explainers. We will use Protodash to summarize the Titanic to a subset (m) of passengers that best represent all of the passengers in the training dataset.

In [ ]:
#df_survived = training_data.query('survived == 1').drop(columns = ['cabin','embarked','name','survived','ticket','fare'])

df_survived = training_data.drop(columns = ['cabin','embarked','name','ticket','fare'])
# convert data to numpy
data = df_survived.to_numpy()
## fillna
df_survived['age'] = df_survived['age'].fillna(-1)
original = df_survived.iloc[1:,:]

prototypes = {}

# one hot encode all features as they are categorical
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(original)

## Applying Protodash

explainer = ProtodashExplainer()

# call Protodash explainer
# S contains indices of the selected prototypes
# W contains importance weights associated with the selected prototypes 

(W, S, _) = explainer.explain(onehot_encoded, onehot_encoded, m=10) 

prototypes={}
prototypes['W']= W
prototypes['S']= S
prototypes['data'] = data
prototypes['original'] = original


In [ ]:
# Display the prototypes along with their computed weights
inc_prototypes = df_survived.iloc[S, :].copy()
# Compute normalized importance weights for prototypes
inc_prototypes["Weights of Prototypes"] = np.around(W/np.sum(W), 2) 
inc_prototypes.transpose()

### ProtoDash to check the prediction

In [ ]:
# Curating the dataset to only include features we are interested in
selected_features = ["survived","pclass","age","sex","sibsp","parch","fare","embarked"]
curated_dataset = pd.get_dummies(df_data_1[selected_features])
# Dropping rows where the age or fare is NaN
curated_dataset.dropna(subset = ["age"], inplace=True)
curated_dataset.dropna(subset = ["fare"], inplace=True)
curated_dataset.head()




In [ ]:

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# Define, Train, and Test a Logistic Regression Classifier

# Splitting the data into training and testing sets
x_features = ['pclass', 'age','sibsp', 'parch', 'fare', 'sex_female', 'sex_male', 'embarked_C', 'embarked_Q', 'embarked_S']
y_feature = ['survived']
x = pd.get_dummies(curated_dataset[x_features])
y = pd.get_dummies(curated_dataset[y_feature])
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size = 0.2, random_state = 0)

# fitting Logistic Regression to the Training set
skl_LR_model = LogisticRegression()
skl_LR_model.fit(x_train, y_train)

# predicting the test result
y_pred = skl_LR_model.predict(x_test)

print("Accuracy score : ", accuracy_score(y_test,y_pred))



In [ ]:


# Creating two more tables from the training set: one of deceased, the other of survivors
grouped = x_train.groupby(curated_dataset.survived)
passengers_alive = grouped.get_group(1)
passengers_dead = grouped.get_group(0)



In [ ]:


# Training Table of Surviving Passengers
passengers_alive.insert(0,'survived',1)
passengers_alive.head()



In [ ]:
# Training Table of Deceased Passengers
passengers_dead.insert(0,'survived',0)
passengers_dead.head()



In [ ]:
# Test set from which we select passengers to predict outcomes for
x_test.head(20)

In [ ]:
# Select a passenger from the Test Set
person_id = 311
x_test.loc[[person_id]]



In [ ]:

# Predict whether the passenger survives
y_pred_outcome = skl_LR_model.predict(x_test.loc[[person_id]])
y_prob = skl_LR_model.predict_proba(x_test.loc[[person_id]])[::,1]
#y_confidence = skl_LR_model.decision_function(x_test.loc[[person_id]])

print("Predicted Outcome :  ", y_pred_outcome)
print("Probability of Survival : ",y_prob)
#print("Confidence Score : ",y_confidence)


In [ ]:
#Now Applying Explainer




# Use Protodash to find instances similar to the passenger

# Attach the prediction made by the model to X
X = x_test.loc[[person_id]].copy()
X.insert(0,'survived',y_pred_outcome)
#X.head()

# convert X and passengers_dead to numpy arrays
X_numpy = X.to_numpy()
passengers_dead_numpy = passengers_dead.to_numpy()

# Find prototype passengers that DID NOT Survive
explainer = ProtodashExplainer()
(W, S, setValues) = explainer.explain(X_numpy, passengers_dead_numpy, m=5) # Return weights W, Prototypes S and objective function values

# Display the Dead prototypes along with their computed weights
dead_prototypes = passengers_dead.iloc[S, :].copy()
# Compute normalized importance weights for prototypes
dead_prototypes["Weights of Prototypes"] = np.around(W/np.sum(W), 2) 
dead_prototypes.head()

In [ ]:

# Find prototype passengers that DID Survive
passengers_alive_numpy = passengers_alive.to_numpy()
(W2, S2, setValues2) = explainer.explain(X_numpy, passengers_alive_numpy, m=5) # Return weights W, Prototypes S and objective function values

# Display the Alive prototypes along with their computed weights
alive_prototypes = passengers_alive.iloc[S2, :].copy()
# Compute normalized importance weights for prototypes
alive_prototypes["Weights of Prototypes"] = np.around(W2/np.sum(W2), 2) 
alive_prototypes.head()

## 3 - Passenger : Why didnt i survive? What can I change to survive?




[<h2>CEM</h2>](#INDEX)

A passenger that had similar traits to one who didnt survive, but that passenger survived. To see how we can better our chances of survival

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential, Model, load_model, model_from_json
from keras.layers import Dense
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

from aix360.algorithms.contrastive import CEMExplainer, KerasClassifier
from aix360.algorithms.contrastive import CEM


In [ ]:
## define train & test


# Map all females to 0 and all males to 1
training_data.sex.replace(to_replace='female', value=0, inplace=True)
training_data.sex.replace(to_replace='male', value=1, inplace=True)
test_data.sex.replace(to_replace='female', value=0, inplace=True)
test_data.sex.replace(to_replace='male', value=1, inplace=True)


x_train = training_data[['sex','pclass','parch','sibsp']]

x_test = test_data[['sex','pclass','parch','sibsp']]

y_train_b = (training_data['survived'])
y_test_b = (test_data['survived'])


x_train.head()

In [ ]:
## normalize

Z = np.vstack((x_train, x_test))
Zmax = np.max(Z, axis=0)
Zmin = np.min(Z, axis=0)


#normalize an array of samples to range [-0.5, 0.5]
def normalize(V):
    VN = (V - Zmin)/(Zmax - Zmin)
    VN = VN - 0.5
    return(VN)
    
# rescale a sample to recover original values for normalized values. 
def rescale(X):
    return(np.multiply ( X + 0.5, (Zmax - Zmin) ) + Zmin)

N = normalize(Z)
xn_train = N[0:x_train.shape[0], :]
xn_test  = N[x_train.shape[0]:, :]




In [ ]:
## DEFINE ARCHITECTURE

# nn with no softmax
def nn_small():
    model = Sequential()
    model.add(Dense(8, input_dim=4, kernel_initializer='normal', activation='relu'))
    model.add(Dense(2, kernel_initializer='normal'))    
    return model    

In [ ]:
### train model

In [ ]:
# Set random seeds for repeatability
np.random.seed(1) 
tf.set_random_seed(2) 

class_names = ['Survived', 'Not Survived']

# loss function
def fn(correct, predicted):
    return tf.nn.softmax_cross_entropy_with_logits(labels=correct, logits=predicted)

# compile and print model summary
nn = nn_small()
nn.compile(loss=fn, optimizer='adam', metrics=['accuracy'])
nn.summary()


# train model or load a trained model
TRAIN_MODEL = True

if (TRAIN_MODEL):             
    nn.fit(xn_train, y_train_b, batch_size=30, epochs=100, verbose=0, shuffle=False)
    nn.save_weights("titanic_nnsmall.h5")     
else:    
    nn.load_weights("titanic_nnsmall.h5")
        

# evaluate model accuracy        
score = nn.evaluate(xn_train, y_train_b, verbose=0) #Compute training set accuracy
#print('Train loss:', score[0])
print('Train accuracy:', score[1])

score = nn.evaluate(xn_test, y_test_b, verbose=0) #Compute test set accuracy
#print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
x_test.head()

In [ ]:
y_test_b[0:20]

## Pertient Negative

In [ ]:


# Some interesting user samples to try: 89, 184, XX
idx = 12

#X = pd.DataFrame(xn_train.iloc[idx]).T.astype(float)
X = xn_test[idx].reshape((1,) + xn_test[idx].shape)

print("Computing PN for Sample:", idx)
print("Prediction made by the model:", nn.predict_proba(X))
print("Prediction probabilities:", class_names[np.argmax(nn.predict_proba(X))])
print("")

mymodel = KerasClassifier(nn)
explainer = CEMExplainer(mymodel)

arg_mode = 'PN' # Find pertinent negatives
arg_max_iter = 500 # Maximum number of iterations to search for the optimal PN for given parameter settings
arg_init_const = 10.0 # Initial coefficient value for main loss term that encourages class change
arg_b = 9 # No. of updates to the coefficient of the main loss term
arg_kappa = 0.2 # Minimum confidence gap between the PNs (changed) class probability and original class' probability
arg_beta = .1 # Controls sparsity of the solution (L1 loss)
arg_gamma = 90 # Controls how much to adhere to a (optionally trained) auto-encoder
my_AE_model = None # Pointer to an auto-encoder
arg_alpha = 0.01 # Penalizes L2 norm of the solution
arg_threshold = 1. # Automatically turn off features <= arg_threshold if arg_threshold < 1
arg_offset = 0 # the model assumes classifier trained on data normalized
                # in [-arg_offset, arg_offset] range, where arg_offset is 0 or 0.5
# Find PN for applicant 89
(adv_pn, delta_pn, info_pn) = explainer.explain_instance(X, 
                                                         arg_mode, 
                                                         my_AE_model, 
                                                         arg_kappa, 
                                                         arg_b,
                                                         arg_max_iter, 
                                                         arg_init_const, 
                                                         arg_beta, 
                                                         arg_gamma,
                                                         arg_alpha, 
                                                         arg_threshold, 
                                                         arg_offset
                                                        )





In [ ]:
Xpn = adv_pn
classes = [ class_names[np.argmax(nn.predict_proba(X))], class_names[np.argmax(nn.predict_proba(Xpn))], 'NIL' ]

print("Sample:", idx)
print("prediction(X)", nn.predict_proba(X), class_names[np.argmax(nn.predict_proba(X))])
print("prediction(Xpn)", nn.predict_proba(Xpn), class_names[np.argmax(nn.predict_proba(Xpn))] )


X_re = rescale(X) # Convert values back to original scale from normalized
Xpn_re = rescale(Xpn)
Xpn_re = np.around(Xpn_re.astype(np.double), 2)

delta_re = Xpn_re - X_re
delta_re = np.around(delta_re.astype(np.double), 2)
delta_re[np.absolute(delta_re) < 1e-4] = 0

X3 = np.vstack((X_re, Xpn_re, delta_re))

dfre = pd.DataFrame.from_records(X3) # Create dataframe to display original point, PN and difference (delta)
dfre[23] = classes

dfre.columns = training_data[['sex','pclass','parch','sibsp','survived']].columns
dfre.rename(index={0:'X',1:'X_PN', 2:'(X_PN - X)'}, inplace=True)
dfret = dfre.transpose()


def highlight_ce(s, col, ncols):
    if (type(s[col]) != str):
        if (s[col] > 0):
            return(['background-color: yellow']*ncols)    
    return(['background-color: white']*ncols)

dfret.style.apply(highlight_ce, col='(X_PN - X)', ncols=3, axis=1) 


In [ ]:


plt.rcdefaults()
fi = abs((X-Xpn).astype('double'))/np.std(xn_train.astype('double'), axis=0) # Compute PN feature importance
objects = training_data[['sex','pclass','parch','sibsp','survived']].columns[-2::-1]
y_pos = np.arange(len(objects))
performance = fi[0, -1::-1]

plt.barh(y_pos, performance, align='center', alpha=0.5) # bar chart
plt.yticks(y_pos, objects) # Display features on y-axis
plt.xlabel('weight') # x-label
plt.title('PN (feature importance)') # Heading

plt.show() # Display PN feature importance



**Pertinent Positive**

In [ ]:
# Some interesting user samples to try: 9 11 24
idx = 12

X = xn_test[idx].reshape((1,) + xn_test[idx].shape)
print("Computing PP for Sample:", idx)
print("Prediction made by the model:", class_names[np.argmax(nn.predict_proba(X))])
print("Prediction probabilities:", nn.predict_proba(X))
print("")


mymodel = KerasClassifier(nn)
explainer = CEMExplainer(mymodel)

arg_mode = 'PP' # Find pertinent positives
arg_max_iter = 1000 # Maximum number of iterations to search for the optimal PN for given parameter settings
arg_init_const = 10.0 # Initial coefficient value for main loss term that encourages class change
arg_b = 9 # No. of updates to the coefficient of the main loss term
arg_kappa = 0.2 # Minimum confidence gap between the PNs (changed) class probability and original class' probability
arg_beta = 10.0 # Controls sparsity of the solution (L1 loss)
arg_gamma = 100 # Controls how much to adhere to a (optionally trained) auto-encoder
my_AE_model = None # Pointer to an auto-encoder
arg_alpha = 0.1 # Penalizes L2 norm of the solution
arg_threshold = 0.0 # Automatically turn off features <= arg_threshold if arg_threshold < 1
arg_offset = 0.5 # the model assumes classifier trained on data normalized
                # in [-arg_offset, arg_offset] range, where arg_offset is 0 or 0.5
(adv_pp, delta_pp, info_pp) = explainer.explain_instance(X, arg_mode, my_AE_model, arg_kappa, arg_b,
                                                         arg_max_iter, arg_init_const, arg_beta, arg_gamma,
                                                            arg_alpha, arg_threshold, arg_offset)



In [ ]:
Xpp = delta_pp
classes = [ class_names[np.argmax(nn.predict_proba(X))], class_names[np.argmax(nn.predict_proba(Xpp))]]

print("PP for Sample:", idx)
print("Prediction(Xpp) :", class_names[np.argmax(nn.predict_proba(Xpp))])
print("Prediction probabilities for Xpp:", nn.predict_proba(Xpp))
print("")

X_re = rescale(X) # Convert values back to original scale from normalized
adv_pp_re = rescale(adv_pp)
#Xpp_re = X_re - adv_pp_re
Xpp_re = rescale(Xpp)
Xpp_re = np.around(Xpp_re.astype(np.double), 2)
Xpp_re[Xpp_re < 1e-4] = 0

X2 = np.vstack((X_re, Xpp_re))

dfpp = pd.DataFrame.from_records(X2.astype('double')) # Showcase a dataframe for the original point and PP
dfpp[23] = classes
dfpp.columns = test_data[['sex','pclass','parch','sibsp','survived']].columns
dfpp.rename(index={0:'X',1:'X_PP'}, inplace=True)
dfppt = dfpp.transpose()

dfppt.style.apply(highlight_ce, col='X_PP', ncols=2, axis=1) 

In [ ]:


plt.rcdefaults()
fi = abs(Xpp_re.astype('double'))/np.std(xn_test.astype('double'), axis=0) # Compute PP feature importance
    
objects = training_data[['sex','pclass','parch','sibsp','survived']].columns[-2::-1]
y_pos = np.arange(len(objects)) # Get input feature names
performance = fi[0, -1::-1]

plt.barh(y_pos, performance, align='center', alpha=0.5) # Bar chart
plt.yticks(y_pos, objects) # Plot feature names on y-axis
plt.xlabel('weight') #x-label
plt.title('PP (feature importance)') # Figure heading

plt.show()    # Display the feature importance



# More Explainers applied - backup feel free to explore

[<h2>TED - INCLUDE EXPLANATIONS IN YOUR MODEL</h2>](#INDEX)


The TED_CartesianExplainer is an implementation of the algorithm in the [AIES'19 paper by Hind et al.](https://arxiv.org/abs/1811.04896) It is most suited for use cases where matching explanations to the mental model of the explanation consumer is the highest priority; i.e., where the explanations are similar to what would be produced by a domain expert.

To achieve this goal, the TED (Teaching Explanations for Decisions) framework requires that the training data is augmented so that each instance contains an explanation (E). The goal is to teach the framework what are appropriate explanations in the same manner the training dataset teaches what are appropriate labels (Y). Thus, the training dataset contains the usual features (X) and labels (Y), augmented with an explanation (E) for each instance. 

The format of the explanation is flexible and determined by the use case. It can be a number, text, an image, an audio, a video, etc. The TED framework simply requires that it can be mapped to a unique integer [0, N] and that any two explanations that are semantically the same should be mapped to the same integer. 

In this titanic example, we would need to create a number of explanations on why specific passengers would not be able to get to a life-saving boat, and map those explanation to a unique integer as follows: 

**Explanation Code / Legend**


| Explanation | code | 
| ---- | ----  |
| Priority to life boat bacause the passenger is a woman with children | 1 |
| Priority to life boat bacause the passenger is first class | 2 |
| No priority to life boat because the passenger is young man with no family belonging to the third class | 3 |
| ... | ...  |


**New Training Set**


Then we should augment the training set with the Explanations like the following: 

| "pclass" | "sex" | "sibsp" | "parch" | "survived" |  "explanation" |
| ---- | ----| ---- | ----| ----|  ---- |
| 1| 0 | 1 | 2| 1|  1 |
| 3| 1 | 0 | 0| 0|  3 |
| ...| ... | ... | ... | ... |  ... |


The new training set contains: features X ("pclass", "sex", "sibsp", "parch"), labels Y ("survived") and the explanation E ("explanations"). In the scoring phase, the TED algorithm now is able to output for a new passenger, a pair of Y (label) and E (Explanbility). The explainability E  will be an integer but it can be easily mapped back to the text using the explanation code/legend. 



[<h2>LOCAL EXPLAINERS - APPLY LIME, SHAP AND CEM TO RANDOM FOREST</h2>](#INDEX)

<span style="text-decoration: underline">Random Forest Classifier</span>

<img src="https://i.imgur.com/AC9Bq63.png" alt="Model Overview" width="900"/>


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Define the features of interest
features_rfc = ["pclass", "sex", "sibsp", "parch"]

# Map all females to 0 and all males to 1
training_data.sex.replace(to_replace='female', value=0, inplace=True)
training_data.sex.replace(to_replace='male', value=1, inplace=True)
test_data.sex.replace(to_replace='female', value=0, inplace=True)
test_data.sex.replace(to_replace='male', value=1, inplace=True)

# Set up X feature matrix and y vectors for the test set and training set
y_train_rfc = training_data["survived"]
X_train_rfc = training_data[features_rfc]
X_test_rfc = test_data[features_rfc]
#X_train_rfc = pd.get_dummies(training_data[features_rfc])
#X_test_rfc  = pd.get_dummies(test_data[features_rfc])
y_test_rfc  = test_data["survived"] 

# Create the classifier and fit the model
rfcModel = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
rfcModel.fit(X_train_rfc, y_train_rfc)

Use the trained classifier (stored in the rfcModel variable) to predict the outcome of each passenger in the test set and print out the results.

In [ ]:
# Get the predictions
predictions_rfc = rfcModel.predict(X_test_rfc)

print('Completed!')

correct_rfc= 0
total_rfc  = len(predictions_rfc)

print("Predicted Actual")
# print(y_test_rfc)

# loop over each prediction
for i in range(total_rfc):
    prediction_rfc = predictions_rfc[i] # an integer with value 0 (did not survive) or 1 (survived)
    actual_rfc = y_test_rfc[i] # the ground truth values for survival of people in the test set
    if (prediction_rfc == actual_rfc): 
        # you got one right, well done!
        correct_rfc += 1
#         print('%9d %d correct!' %(prediction_rfc,actual_rfc))
#     else:
#         # can you work out why your classifier was wrong here?
#         print('%9d %d' %(prediction_rfc,actual_rfc))


print('\n')
print('Total number correct :', correct_rfc)
print('Total number of tests:', total_rfc)
print('Accuracy percentage  :  %f' %(correct_rfc/total_rfc*100))


### Local explanations - SHAP & Lime


After obtaining the prediction with a Random Forest Classifier, we can now apply different explainers to the obtained predictions. We will start with SHAP and LIME, which are local explainers. This means that they provide explanations for each datapoint in our dataset. 



[<h2>LIME</h2>](#INDEX)

In [ ]:
from IPython.display import clear_output
from aix360.algorithms.lime import LimeTabularExplainer

LIME takes as input the output classes *[Will survive, will not survive]*, the model features, the input values, and the model predictions.
It is also important to define upfront a random state, to assure replicability of the results. It is also possible to define the numebr of samples that the model considers as neighbours to the observation we want to explain.

In [ ]:
explainerLIME = LimeTabularExplainer(training_data = X_train_rfc.values, 
                                     feature_names=X_train_rfc.columns.values.tolist(), 
                                     class_names=['Will Die','Will Survive'], 
                                     verbose=True)


# predict_fn_rf = lambda x: rfcModel.predict_proba(x).astype(float)
# X_rfc = X_train_rfc.values
# explainerLIME = lime.lime_tabular.LimeTabularExplainer(X_rfc,feature_names = X_train_rfc.columns,class_names=['Will Die','Will Survive'],kernel_width=5, random_state = 123456)


Once traine the explainer, we can choose a passenger of interest and apply the *explainerLIME.explain_instance* method to understand how much each feature influenced the final results.
We can change the person_id below to choose a different passenger from the test or training file (0-417)

In [ ]:
person_id = 0
X_test_rfc.loc[[person_id]]

In [ ]:
person = X_test_rfc.loc[[person_id]].values[0]
exp = explainerLIME.explain_instance(person, rfcModel.predict_proba ,num_features=10)
exp.show_in_notebook(show_all=False)

### COMPARING LIME PREDICTIONS ACROSS DIFFERENT PASSENGERS

One of LIME limitations, is that it can provide different explanations for same inputs.

Let's consider, for example, a person with the following characteristics:
- pclass = 1
- sex = 3
- sibsp = 0 
- parch = 0

Observations with id 0, 10, and 17 in the test dataset, all share this same configuration.
Below we apply the same LIMEexplainer to these three passengers - Kelly, Illief and Assaf.

In [ ]:
test_data.iloc[[0,10,17]]

In [ ]:
## passenger 1 explanation
person_test = X_test_rfc.loc[[0]].values[0]
exp_test = explainerLIME.explain_instance(person_test, rfcModel.predict_proba,num_samples=100,num_features=10)
passenger_1 = pd.DataFrame(exp_test.as_list()).rename(columns={0:'FEATURE',1:'PASSENGER_892'})
## passenger 2 explanation
person_test = X_test_rfc.loc[[10]].values[0]
exp_test = explainerLIME.explain_instance(person_test, rfcModel.predict_proba,num_samples=100,num_features=10)
passenger_2 = pd.DataFrame(exp_test.as_list()).rename(columns={0:'FEATURE',1:'PASSENGER_902'})
## passenger 3 explanation
person_test = X_test_rfc.loc[[17]].values[0]
exp_test = explainerLIME.explain_instance(person_test, rfcModel.predict_proba,num_samples=100,num_features=10)
passenger_3 = pd.DataFrame(exp_test.as_list()).rename(columns={0:'FEATURE',1:'PASSENGER_909'})
## compare overall results
overall = pd.merge(passenger_1, pd.merge(passenger_2, passenger_3, how = 'inner', on = 'FEATURE'), how = 'inner')

for i in range(10):
    clear_output(wait=True)
    
overall

**RESULTS**

Persons with same parameters have (slightly) different results - in some cases thet also change their sign (parch/sibsp). Thus, if we want to use LIME to provide explanations to final users, it is not possible to show a single number; we need to find a summary measure. 
A possibility, for example, is to compute average, standard deviation and build confidence interval for every unique combination of parameters.

### PERMUTING OBSERVATIONS ORDER: IMPACT ON EXPLANATIONS

What if we do change the order of observations in the training data before applying LIME .. does impact the final result? Yes, it does (let's see what happens for instance for passengers_id 892, 897, 902, 909.

In [ ]:
## get LIME PREDICTION FOR DIFFERENT PASSENGERS WITH lime - passenger = [3,1,0,0] (pclass, sex, sibsp, parch)
## in this first test we find lime explanations for 4 passengers, ids = [[909,902,892,897]] from X_TEST_RFC. 
## results are displayed below

X_test_rfc
passengers = [0,5,10,17]

results = pd.DataFrame()
for p in passengers:
    person_test = X_test_rfc.loc[[p]].values[0]
    exp_test = explainerLIME.explain_instance(person_test, rfcModel.predict_proba, num_samples=100,num_features=10)
    dfp = pd.DataFrame(exp_test.as_list())
    dfp['ID'] = test_data.loc[[p]].passengerid.values[0]
    results = pd.concat([results,dfp])

results_no_order = results
results_no_order = results_no_order.rename(columns = {0:'RULE',1:'WEIGHT_NO_ORDER'})

for i in range(10):
    clear_output(wait=True)
    print("Done!")

In [ ]:
## get LIME PREDICTION FOR DIFFERENT PASSENGERS WITH lime - passenger = [3,1,0,0] (pclass, sex, sibsp, parch)
## in this first test we find lime explanations for 4 passengers, ids = [[909,902,892,897]] from X_TEST_RFC_ORDER.
## before computing LIME explanations, we reorder the dataset so that the passengers are in reversed order
## even though the relative weight of the different features remains the same, the exact value differs from
## previous test. Also Rules are a little different from before. In some cases, the sign of features also varies

test_data_order = test_data.sort_values(by = 'name').reset_index(drop=True)

X_test_rfc_order = test_data_order[['pclass','sex','sibsp','parch']]
passengers = [21,187,206,366]

results = pd.DataFrame()
for p in passengers:
    person_test = X_test_rfc_order.loc[[p]].values[0]
    exp_test = explainerLIME.explain_instance(person_test, rfcModel.predict_proba,num_samples=100,num_features=10)
    dfp = pd.DataFrame(exp_test.as_list())
    dfp['ID'] = test_data_order.loc[[p]].passengerid.values[0]
    results = pd.concat([results,dfp])

results_order = results
results_order = results_order.rename(columns = {0:'RULE',1:'WEIGHT_ORDER'})

for i in range(10):
    clear_output(wait=True)
    print("Done!")

In [ ]:
## merge two datasets to compare differences
compare_lime = pd.merge(results_order, results_no_order, on = ['ID', 'RULE'])
compare_lime['DELTA'] = compare_lime['WEIGHT_ORDER'] - compare_lime['WEIGHT_NO_ORDER']
compare_lime

**RESULTS**

Each same passenger ID, in the two datasets (WEIGHT_ORDER and WEIGHT_NO_ORDER) did get slightly varying features weights after changing the input order. This is due to the fact that LIME samples observation around the point that need to be explained. Thus, changing the order also impact the neighbourhood of each datapoint.


### COMPUTE AVERAGE FEATURE WEIGHTS ACROSS DATASET

Given that LIME does not provide a unique explanations (slightly different weights for equal input), we will try to compute the average/std/min/max/count for each unique observation in the test dataset. 
This might give a more accurate idea of the weight of each features.


In [ ]:
## STEP 1 - UNIQUE ARRAY OF FEATURES

passengers = X_test_rfc.copy()
passengers = passengers.groupby(['pclass','sex','sibsp','parch']).count().reset_index(drop = False)
passengers = passengers.reset_index()

## STEP 2 - FOR EACH PASSENGER, CREATE LIME EXPLANATIONS
features = ['pclass','sex','sibsp','parch']
explanations = pd.DataFrame()

def get_explanations(person_test, predict_fn_rf, features):
    #person_test = df.loc[[p]][[features]].values[0]
    exp_test = explainerLIME.explain_instance(person_test, rfcModel.predict_proba,num_samples=100,num_features=10)
    dfp = pd.DataFrame(exp_test.as_list())
    dfp = dfp.join(pd.DataFrame(person_test).T.reset_index(drop = True), how = 'outer').ffill()
    #dfp['ID'] = df.loc[[p]].passengerid.values[0]
    return pd.concat([explanations, dfp])


new_df = X_test_rfc.apply(lambda x: get_explanations(x,rfcModel.predict_proba,features),1)
results = pd.concat(dict(new_df)).reset_index(drop = True)
results = results.rename(columns = {0:'rule',1:'weight'})
#results['feature'] = results['rule'].apply(lambda x: features[i] if (features[i] in str(x)) else None for i in features)
results = pd.merge(results, passengers, on = ['pclass','sex','sibsp','parch'])

## rules are the same for each unique passenger

## STEP 3 - for each passenger, compute mean, std
features_importance = results.groupby(['index','rule']).agg({'weight':['mean','std','count','min','max']}).fillna(0)


## STEP 4 - GRAPH
features_importance.head()

## STEP 5 - OVERALL MIN/MAX for each feature

index1 = features_importance.copy()
index1.columns = index1.columns.droplevel(0)
final_result = index1.reset_index().groupby('rule').agg({'min':'min','max':'max'})


for i in range(10):
    clear_output(wait=True)

final_result

**CONSIDERATIONS**

In the provided features matrix, we can notice that some variables - parch and sibsp - have weights that can be positive or negative based. Thus, their overall contribution on the prediction is uncertain (in a linear model a feature with lower and upper confidence interval values with different are not even significant). Conversely, all rules involving sex and pclass have consistent sign across all observations.

[<h2>SHAP</h2>](#INDEX)

We now train a SHAP classifier and compute explanations for the predictions.
Here we use TreeExplainer, which is one explainer that can be applied to Tree-based models. SHAP also provides explainers for linear models, agnostic explainers (ex. Permutations, Partition), GPU-based explainers (GPU Tree).

SHAP Documentation: https://shap.readthedocs.io/en/latest/index.html

In [ ]:
# Importing shap KernelExplainer (aix360 style)
from aix360.algorithms.shap import TreeExplainer

# the following import is required for access to shap plotting functions and datasets
import shap
shap.initjs()


## AIX360 
explainer = TreeExplainer(rfcModel)
# shap_values = TreeExplainer(rfcModel).shap_values(X_train_rfc)
# shap_values_test = TreeExplainer(rfcModel).shap_values(X_test_rfc)


## ORIGINAL PACKAGE ALLOWS TO COMPUTE ALL THE SHAP VALUES AT ONCE
# import shap
# explainer = shap.TreeExplainer(rfcModel)
# shap_values = shap.TreeExplainer(rfcModel).shap_values(X_train_rfc)
# shap_values_test = shap.TreeExplainer(rfcModel).shap_values(X_test_rfc)


In [ ]:
person_id = 12
shap_values = explainer.explain_instance(X_test_rfc.loc[[person_id]].values[0])
shap.force_plot(explainer.explainer.expected_value[1], shap_values[1], X_test_rfc.iloc[[person_id]])


#test if person with similar profile get the same result - ORIGINAL PACKAGE
# person_id = 0
# shap.initjs()
# shap.force_plot(explainer.expected_value[1], shap_values_test[1][person_id], X_test_rfc.iloc[[person_id]])


We can now apply the SHAP explainer to different observations. The graph visually dipslays the weight of each feature on the final prediction.

In [ ]:
#test if person with similar profile get the same result
person_id = 2
shap_values = explainer.explain_instance(X_test_rfc.loc[[person_id]].values[0])
shap.force_plot(explainer.explainer.expected_value[1], shap_values[1], X_test_rfc.iloc[[person_id]])


In [ ]:
#test if person with similar profile get the same result
person_id = 17
shap_values = explainer.explain_instance(X_test_rfc.loc[[person_id]].values[0])
shap.force_plot(explainer.explainer.expected_value[1], shap_values[1], X_test_rfc.iloc[[person_id]])



Let's perform now the same test as for LIME explainer. We identify equal input rows (same passengers) and we apply the SHAP explainers to them.
SHAP will provide the same results for the same inputs. This assure higher reproducibility of results if compared to LIME.

**COMPARE LIME AND SHAP**

We would like to understand whether the explanations provided by SHAP and LIME are similar for each passenger.
Give that those explainers provide weights in different formats (rules for LIME, features for LIME), we will concentrate on Find instances of the data for which the 
- ranking
- sign
instead of single weights for each feature. 
The comparison is computed for each single person, even though LIME returns different explanations for equal rows in the dataset (same input, different output). At the end, we compare in which % of observations feature ranking and feature sign differ using LIME or SHAP.

In [ ]:
## STEP 1 - BUILD DF WITH SHAP EXPLANATIONS FOR ALL OBSERVATIONS
## SHAP_VALUES[1] --> PROBABILITY OF SURVIVAL
overall_values = []

for row_id in range(X_test_rfc.shape[0]):
    overall_values.append(explainer.explain_instance(X_test_rfc.iloc[[row_id]].values[0])[1])
    
shap_df = pd.DataFrame(overall_values, columns = ['pclass','sex','sibsp','parch']).reset_index()
shap_df = shap_df.melt(id_vars = ['index']).sort_values(by =['index','value'], key = abs).reset_index(drop=True)
shap_df['segno_SHAP'] = shap_df.value.apply(lambda x: 1 if x>= 0 else 0)
shap_df = shap_df.rename(columns = {'value':'weight_SHAP','variable':'variable_SHAP'})

## STEP 2 - BUILD DF WITH LIME EXPLANATIONS
features = ['pclass','sex','sibsp','parch'] 
explanations = pd.DataFrame()

def get_explanations(person_test, predict_fn_rf, index):
    exp_test = explainerLIME.explain_instance(person_test, predict_fn_rf,num_samples=100,num_features=10)
    dfp = pd.DataFrame(exp_test.as_list())
    dfp = dfp.join(pd.DataFrame(person_test).T.reset_index(drop = True), how = 'outer').ffill()
    dfp['index'] = index
    return pd.concat([explanations, dfp])

predict_fn_rf = lambda x: rfcModel.predict_proba(x).astype(float)
new_df = X_train_rfc.apply(lambda x: get_explanations(x,predict_fn_rf, x.name),1)
results = pd.concat(dict(new_df)).reset_index(drop = True)
results = results.rename(columns = {0:'rule',1:'weight'})
results['variable_LIME'] = results['rule'].apply(lambda x: [i for i in features if i in str(x)][0])
results['segno_LIME'] = results.weight.apply(lambda x: 1 if x >= 0 else 0)
results = results.sort_values(by = ['index','weight'], key = abs).rename(columns ={'weight':'weight_LIME'}).reset_index(drop=True)

## STEP 3 - Compare order

## compare variable order
compare_shap_lime = pd.merge(shap_df, results[['index','variable_LIME','weight_LIME','segno_LIME']], right_index=True, left_index=True)
compare_shap_lime['order_difference'] = compare_shap_lime['variable_LIME'] == compare_shap_lime['variable_SHAP']

for i in range(10):
    clear_output(wait=True)

print("% of observation for which the order of the most important variables is different")
print(pd.DataFrame(compare_shap_lime.groupby('index_x').sum()['order_difference']).query("order_difference <4").shape[0]/X_train_rfc.shape[0])


In [ ]:
## STEP 3 - Compare order

#compare_shap_lime = pd.merge(shap_df, results[['index','variable','weight_LIME','segno_LIME']], on = ['index','variable'])
compare_sign_difference = pd.merge(shap_df, results[['index','variable_LIME','weight_LIME','segno_LIME']], right_on = ['index','variable_LIME'], left_on = ['index','variable_SHAP'],)
compare_sign_difference['sign_differences'] = compare_sign_difference['segno_LIME'] == compare_sign_difference['segno_SHAP']
print("% of observation for which the sign of the most important variables is different")
print(pd.DataFrame(compare_sign_difference.groupby('index').sum()['sign_differences']).query("sign_differences <4").shape[0]/X_train_rfc.shape[0])


# BACKUP - OTHER MODELS

# <span style="text-decoration: underline">Logistic Regression Classifier</span>

## Function

A function is defined that does the following:
- reads the titanic test or training data
- applies a few cleaning measures to the data
- transforms some data to make it easier to work with

It returns the following to train the classifier or test/score the predictions
- X matrix 
- y vector

Other approaches can be take with the data, this is just an example

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
def get_X_and_y(data):


    # Map all females to 0 and all males to 1
    data.sex.replace(to_replace='female', value=0, inplace=True)
    data.sex.replace(to_replace='male', value=1, inplace=True)

    # Map all NaN ages to -1
    data.age.fillna(-1, inplace=True)

    # Map all NaN fares to -1
    data.fare.fillna(-1, inplace=True)

    # Map the ports of embarkation to numbers
    # C = Cherbourg; Q = Queenstown; S = Southampton
    data.embarked.fillna(-1, inplace=True)
    data.embarked.replace(to_replace='C', value=0, inplace=True)
    data.embarked.replace(to_replace='Q', value=1, inplace=True)
    data.embarked.replace(to_replace='S', value=2, inplace=True)

    # Set up the feature matrix
    X = pd.concat([
        data.pclass,
        data.sex,
        #data.age,
        data.sibsp,
        data.parch,
        #data.fare,
        #data.embarked
    ], axis=1)

    # Set up the vector of supervised learning labels
    try:
        y = data.survived
    except AttributeError:
        y = None

    return X, y

## Create, Train and Run The Model

The data is loaded above and this calls the get_X_and_y() function and feeds the resulting values to train a machine learning classification model.

In [ ]:
# Set up X feature matrix and y vectors for the test set and training set
X_train_lrc, y_train_lrc= get_X_and_y(training_data)
X_test_lrc,  y_test_lrc   = get_X_and_y(test_data)

# Fit the model
lrcModel = LogisticRegression(solver='liblinear').fit(X_train_lrc, y_train_lrc)

Use the trained classifier (stored in the lrcModel variable) to predict the outcome of each passenger in the test set and print out the results.

In [ ]:
# Use the model fitted in our classifier above to predict all rows in the test set
predictions = lrcModel.predict(X_test_lrc)

# Define some useful variables we'll use for calculations later
correct = 0
total = len(predictions)

print("Predicted Actual")
print(y_test_lrc)

# Loop over each prediction
for i in range(total):
    # Integer with value 0 (did not survive) or 1 (survived)
    prediction = predictions[i] 
    # Ground truth values for survival of people in the test set
    actual = y_test_lrc[i] 
    if (prediction == actual): 
        # Correct! Well Done!
        correct += 1
        print('%9d %d correct!' %(prediction,actual))
    else:
        # Incorrect - model needs a tweak
        print('%9d %d' %(prediction,actual))


print('\n')
print('Total number correct :', correct)
print('Total number of tests:', total)
print('Accuracy percentage  :  %f' %(correct/total*100))

**SUBMODULAR PICK - LIME** --> TO BE COMPLETED

Taken from: https://towardsdatascience.com/decrypting-your-machine-learning-model-using-lime-5adc035109b5

LIME aims to attribute a model’s prediction to human-understandable features. In order to do this, we need to run the explanation model on a diverse but representative set of instances to return a nonredundant explanation set that is a global representation of the model



In [ ]:
# Code for SP-LIME
import warnings
from lime import submodular_pick

# Remember to convert the dataframe to matrix values
# SP-LIME returns exaplanations on a sample set to provide a non redundant global decision boundary of original model
sp_obj = submodular_pick.SubmodularPick(explainerLIME, X_train_rfc[['pclass','sex','sibsp','parch']].values, \
predict_fn_rf, num_features=5,num_exps_desired=10)



TOOK 15 MIN TO COMPUTE (MORE OR LESS) --> EMPTY EXPLANATION SET ... **NEED TO UNDERSTAND WHY????**

In [ ]:
[exp.as_pyplot_figure(label=1) for exp in sp_obj.sp_explanations]

In [ ]:
sp_obj.sp_explanations

**CEM WITH ALIBI IMPLEMENTATION--> need a different environment** 

Find minimim sufficient number of variables

Tutorial Available here: 

https://docs.seldon.io/projects/alibi/en/stable/examples/cem_iris.html

CEM applied to IRIS dataset

*personal consideration*: CEM seems to be more used with neural networks than traditional machine learning algo

In [ ]:
import tensorflow as tf
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.datasets import load_iris

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False

In [ ]:
feature_names = ['sex','pclass','parch','sibsp']
class_names = ['survived','not survived']

In [ ]:
x_train,y_train = training_data[['sex','pclass','parch','sibsp']], training_data['survived']
x_test, y_test = test_data[['sex','pclass','parch','sibsp']], test_data['survived']
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
def lr_model():
    x_in = Input(shape=(4,))
    x_out = Dense(2, activation='softmax')(x_in)
    lr = Model(inputs=x_in, outputs=x_out)
    lr.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return lr


In [ ]:
lr = lr_model()
lr.summary()
lr.fit(x_train, y_train, batch_size=16, epochs=500, verbose=0)
#lr.save('iris_lr.h5', save_format='h5')



*Contratistive explanation with pertinent negative*

In [ ]:
idx = 0
X = pd.DataFrame(x_test.iloc[idx]).T
print('Prediction on instance to be explained: {}'.format(class_names[np.argmax(lr.predict(X))]))
print('Prediction probabilities for each class on the instance: {}'.format(lr.predict(X)))


*CEM parameters*

*personal consideration*: many parameters to search for

In [ ]:
mode = 'PN'  # 'PN' (pertinent negative) or 'PP' (pertinent positive)
## from article
shape = (1,) + x_train.shape[1:]  # instance shape
kappa = .2  # minimum difference needed between the prediction probability for the perturbed instance on the
            # class predicted by the original instance and the max probability on the other classes
            # in order for the first loss term to be minimized
beta = .1  # weight of the L1 loss term
c_init = 10.  # initial weight c of the loss term encouraging to predict a different class (PN) or
              # the same class (PP) for the perturbed instance compared to the original instance to be explained
c_steps = 10  # nb of updates for c
max_iterations = 1000  # nb of iterations per value of c
feature_range = (x_train.min(axis=0)#.reshape(shape)-.1,  # feature range for the perturbed instance
                 ,x_train.max(axis=0)#.reshape(shape)+.1
                )  # can be either a float or array of shape (1xfeatures)
clip = (-1000.,1000.)  # gradient clipping
lr_init = 1e-2  # initial learning rate

In [ ]:
from alibi.explainers import CEM

*Generate Pertinent Negative*

In [ ]:
# define model
#lr = load_model('iris_lr.h5')

# initialize CEM explainer and explain instance
cem = CEM(lr,mode, 
                   shape, 
                   kappa=kappa, 
                   beta=beta, 
                   feature_range=feature_range,
                    max_iterations=max_iterations, 
                   c_init=c_init, c_steps=c_steps,
                  learning_rate_init=lr_init, clip=clip
                  )
cem.fit(x_train.values, no_info_type='median')  # we need to define what feature values contain the least
                                         # info wrt predictions
                                         # here we will naively assume that the feature-wise median
                                         # contains no info; domain knowledge helps!
explanation = cem.explain(X, verbose=False)

In [ ]:
print(f'Original instance: {explanation.X}')
print('Predicted class: {}'.format(class_names[explanation.X_pred]))

In [ ]:
print(f'Pertinent negative: {explanation.PN}')
print('Predicted class: {}'.format(class_names[explanation.PN_pred]))

*Store explanations for later on*

In [ ]:
expl = {}
expl['PN'] = explanation.PN
expl['PN_pred'] = explanation.PN_pred